In [241]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [242]:
#I adopted the VPG from here and modified to fit our case
#https://gist.github.com/sharavsambuu/28db9cd1197f2a4727c300bd0efdb2d4
import numpy as np
import gym
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import collections

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.2, shape=shape)
    return tf.Variable(initial)

In [243]:
import kmc_env
import kmcsim
from kmcsim.sim import KMCModel
from kmcsim.sim import EventTree
from kmcsim.sim import RunSim
from kmc_env.envs.kmcsim_state_funcs import make_surface_proj,calc_roughness,get_state_reward,get_incremented_rates,gaussian
from kmc_env.envs.kmc_env import *

In [244]:
box = [16, 32, 4]
box_extension=32
target_roughness=0.98
EPISODES=150
wdir=r'C:\Users\ni1\Documents\RL\kmcsim\data\working'
env = KmcEnv(box=box,box_extension=box_extension,target_roughness=target_roughness,
             reward_type='gaussian',reward_multiplier=1000,reward_tolerance=2,
             rates_spread=0.1,rates_adjustment=1,folder_with_params=wdir)
state,reward = env.reset()
state_size = env.state.shape
action_size = (27,)

In [245]:
dim_actions=action_size[0]
dim = env.state.flatten().shape[0]

In [291]:
box = [16, 32, 4]
box_extension=32
target_roughness=0.98
EPISODES=150
wdir=r'C:\Users\ni1\Documents\RL\kmcsim\data\working'
env = KmcEnv(box=box,box_extension=box_extension,target_roughness=target_roughness,
             reward_type='gaussian',reward_multiplier=1000,reward_tolerance=2,
             rates_spread=0.1,rates_adjustment=1,folder_with_params=wdir)
state,reward = env.reset()
state_size = env.state.shape
# action_size = (27,)
dim=env.state.shape[0]*env.state.shape[1]
dim_actions=27

In [ ]:
#Initial state and NN

num_nodes = 100

num_gradients = 1
maxsteps = 1000
num_runs = 1000

sess = tf.InteractiveSession()

state = tf.placeholder(tf.float32, shape=[None, dim])
action_choice = tf.placeholder(tf.float32, shape=[None, dim_actions])
reward_signal = tf.placeholder(tf.float32, shape=(None,1) )
n_timesteps = tf.placeholder(tf.float32, shape=())


W1 = weight_variable([dim, num_nodes])
b1 = bias_variable([num_nodes])
a1 = tf.nn.relu(tf.matmul(state, W1) + b1)

Wo = weight_variable([num_nodes, dim_actions])
bo = bias_variable([dim_actions])
ao = tf.nn.softmax(tf.matmul(a1, Wo) + bo)


log_prob = tf.log(tf.diag_part(tf.matmul(ao, tf.transpose(action_choice))))# fix this so it doesn't need diag
log_prob = tf.reshape(log_prob, (1,-1))
loss = tf.matmul(log_prob, reward_signal)
loss = -tf.reshape(loss, [-1])
train_step = tf.train.AdamOptimizer().minimize(loss)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)


timestep_learning = np.zeros((num_runs,1))
for run in range(num_runs):

    states = np.zeros((maxsteps,dim), dtype='float32')
    actions = np.zeros((maxsteps,dim_actions), dtype='float32')
    rewards = np.zeros((maxsteps,1), dtype='float32')
    timestep =0
    observation,_ = env.reset()
    observation = np.reshape(observation,(1,dim))
    done = False
    
    while not done and timestep < maxsteps:
        if run % 50 == 0:
            env.render()
        action_prob = sess.run(ao, feed_dict={state: observation})
        action = np.argmax(np.random.multinomial(1, action_prob[0]-0.0001))
        new_observation, reward, done = env.step(action)
        
        states[timestep, :] = observation
    
        actions[timestep, action] = 1
        rewards[timestep, :] = reward
        timestep += 1
        new_observation = np.reshape(new_observation,(1,dim))
        observation[:] = new_observation
        print('Timestep: {}, reward: {:.4}, RMS: {:.3}'.format(timestep,reward,calc_roughness(observation)))
    states = states[:timestep, :]
    actions = actions[:timestep, :]
    rewards = rewards[:timestep,:]
    rewards[:, 0] = np.cumsum(rewards[::-1])[::-1]


    for i in range(num_gradients):
        sess.run(train_step, feed_dict={state: states, action_choice: actions, reward_signal: rewards, n_timesteps: timestep})
        print('VPG is running!')
    timestep_learning[run] = timestep
    


plt.plot(timestep_learning)
plt.show()



C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Timestep: 1, reward: 130.8, RMS: 0.819
Timestep: 2, reward: 88.07, RMS: 0.804
Timestep: 3, reward: 173.1, RMS: 0.83
Timestep: 4, reward: 63.06, RMS: 0.792
Timestep: 5, reward: 15.89, RMS: 0.751
Timestep: 6, reward: 37.97, RMS: 0.776
Timestep: 7, reward: 61.74, RMS: 0.792
Timestep: 8, reward: 12.87, RMS: 0.746
Timestep: 9, reward: 28.75, RMS: 0.768
Timestep: 10, reward: 28.75, RMS: 0.768
VPG is running!
Timestep: 1, reward: 628.5, RMS: 1.06
Timestep: 2, reward: 562.3, RMS: 1.07
Timestep: 3, reward: 209.4, RMS: 1.12
Timestep: 4, reward: 112.9, RMS: 1.15
Timestep: 5, reward: 478.9, RMS: 1.08
Timestep: 6, reward: 662.0, RMS: 1.05
Timestep: 7, reward: 556.7, RMS: 1.07
Timestep: 8, reward: 570.5, RMS: 1.06
Timestep: 9, reward: 579.8, RMS: 1.06
Timestep: 10, reward: 579.8, RMS: 1.06
VPG is running!
Timestep: 1, reward: 97.18, RMS: 0.808
Timestep: 2, reward: 13.59, RMS: 0.747
Timestep: 3, reward: 1.676, RMS: 0.705
Timestep: 4, reward: 1.563, RMS: 0.704
Timestep: 5, reward: 0.8379, RMS: 0.696
T

Timestep: 4, reward: 890.6, RMS: 0.942
Timestep: 5, reward: 987.4, RMS: 0.968
Timestep: 6, reward: 906.2, RMS: 0.945
Timestep: 7, reward: 892.6, RMS: 0.942
Timestep: 8, reward: 841.7, RMS: 0.933
Timestep: 9, reward: 667.9, RMS: 0.908
Timestep: 10, reward: 667.9, RMS: 0.908
VPG is running!
Timestep: 1, reward: 9.709, RMS: 1.22
Timestep: 2, reward: 0.6807, RMS: 1.27
Timestep: 3, reward: 23.82, RMS: 1.2
Timestep: 4, reward: 139.6, RMS: 1.14
Timestep: 5, reward: 31.61, RMS: 1.19
Timestep: 6, reward: 129.4, RMS: 1.14
Timestep: 7, reward: 114.3, RMS: 1.15
Timestep: 8, reward: 296.2, RMS: 1.1
Timestep: 9, reward: 667.6, RMS: 1.05
Timestep: 10, reward: 667.6, RMS: 1.05
VPG is running!
Timestep: 1, reward: 928.8, RMS: 1.01
Timestep: 2, reward: 734.4, RMS: 1.04
Timestep: 3, reward: 398.2, RMS: 1.09
Timestep: 4, reward: 504.3, RMS: 1.07
Timestep: 5, reward: 904.3, RMS: 1.02
Timestep: 6, reward: 990.5, RMS: 0.99
Timestep: 7, reward: 964.8, RMS: 0.959
Timestep: 8, reward: 678.8, RMS: 0.91
Timestep: